In [76]:
# === Librerías básicas ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# === Preprocesamiento, modelo y evaluación ===
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [77]:
# Carga individual
df_primera = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-primera-5-cambios.csv', sep=",")
df_premier = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-premier-5-cambios.csv', sep=",")
df_bundesliga = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-bundesliga-5-cambios.csv', sep=",")

# Unir en un solo DataFrame
df = pd.concat([df_primera, df_premier, df_bundesliga], ignore_index=True)

# === Eliminación de columnas irrelevantes o identificadores ===
df.drop(['jornada', 'temporada', 'id_indicadores_equipo_prepartido', 'id_partido'], axis=1, inplace=True)

# === Variables predictoras ===
X = df.drop(['resultado_partido', 'resultado_local', 'resultado_visitante'], axis=1)

# === Variables objetivo ===
y_resultado = df['resultado_partido']           # Para clasificación (1, X, 2)
y_goles_local = df['resultado_local']           # Para regresión
y_goles_visitante = df['resultado_visitante']   # Para regresión


In [78]:
# === Escalado de variables ===
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# === División para clasificación del resultado ===
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_scaled, y_resultado, test_size=0.2, random_state=60
)

# === División para predicción de goles locales ===
X_train_loc, X_test_loc, y_train_loc, y_test_loc = train_test_split(
    X_scaled, y_goles_local, test_size=0.2, random_state=42
)

# === División para predicción de goles visitantes ===
X_train_vis, X_test_vis, y_train_vis, y_test_vis = train_test_split(
    X_scaled, y_goles_visitante, test_size=0.2, random_state=42
)


### Predicciones resultado

In [79]:
# === Modelo base ===
svm_model = SVC()

# === Grid de hiperparámetros ===
param_grid = {
    'C': [0.1, 1, 10],                # Regularización: cuanto mayor, menos margen de error
    'kernel': ['linear', 'rbf'],      # Tipos de kernel: lineal o radial (no lineal)
    'gamma': ['scale', 'auto']        # Sólo aplica al kernel RBF: controla forma del margen
}

# === Búsqueda de hiperparámetros con validación cruzada ===
grid_search_svm = GridSearchCV(
    estimator=svm_model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Entrenamiento
grid_search_svm.fit(X_train_cls, y_train_cls)

# Mostrar mejores parámetros
print("Mejores hiperparámetros para SVM (clasificación):")
print(grid_search_svm.best_params_)


Mejores hiperparámetros para SVM (clasificación):
{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}


In [80]:
# === Entrenamiento final con los mejores parámetros encontrados ===
best_svm_model = grid_search_svm.best_estimator_
best_svm_model.fit(X_train_cls, y_train_cls)

# === Predicción sobre el conjunto de prueba ===
y_pred_cls = best_svm_model.predict(X_test_cls)


In [81]:
# === Evaluación del rendimiento del modelo SVM ===
accuracy = accuracy_score(y_test_cls, y_pred_cls)
print(f"Accuracy del modelo SVM en el conjunto de prueba: {accuracy:.4f}")

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test_cls, y_pred_cls))


Accuracy del modelo SVM en el conjunto de prueba: 0.4576
Matriz de confusión:
[[143  31   4]
 [ 60  14   7]
 [ 78  12   5]]


In [82]:
from sklearn.inspection import permutation_importance

# Obtener los nombres de las variables predictoras
feature_names = X.columns

# Para KNN, la importancia de variables no es directa, pero podemos usar Permutation Importance

# Calculamos la importancia por permutación sobre el conjunto de test
result = permutation_importance(best_svm_model, X_test_cls, y_test_cls, n_repeats=1, random_state=42, n_jobs=-1)

# Obtenemos las 10 variables más importantes y sus importancias
importances = result.importances_mean
indices = np.argsort(importances)[::-1][:10]

print("Top 10 variables más importantes y sus pesos:")
for idx in indices:
    print(f"{feature_names[idx]}: {importances[idx]:.4f}")

# Filtramos las variables relacionadas con sustituciones
sustitucion_mask = feature_names.str.contains('cambio', case=False)
all_mask = feature_names.str.contains('e', case=False)
sustitucion_importance = abs(importances[sustitucion_mask]).sum()
all_importance = abs(importances[all_mask]).sum()
print()

print(f"\Porcentaje de importancia de variables relacionadas con sustituciones: {sustitucion_importance/all_importance:.4f}")

Top 10 variables más importantes y sus pesos:
proporcion local corners a favor en sitio: 0.0056
proporcion visitante cambios 61 a 75 en general: 0.0028
proporcion local cambios centrocampistas a delanteros en general: 0.0028
proporcion local posesion en sitio: 0.0028
proporcion local cambios en sitio: 0.0028
proporcion local cambios defensas a delanteros sitio: 0.0028
proporcion local cambios delanteros a centrocampistas sitio: 0.0028
porcentaje visitante mas 2,5 marcados en sitio: 0.0028
proporcion visitante cambios 45 a 60 en general: 0.0028
proporcion local cambios centrocampistas a defensas sitio: 0.0028

\Porcentaje de importancia de variables relacionadas con sustituciones: 0.3767


### Predicción equipo local

In [83]:
from sklearn.svm import SVR

# === Modelo base para regresión ===
svr_model = SVR()

# === Grid de hiperparámetros ===
param_grid_svr = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# === Búsqueda de hiperparámetros con validación cruzada ===
grid_search_svr = GridSearchCV(
    estimator=svr_model,
    param_grid=param_grid_svr,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

# Entrenar sobre datos de goles del equipo local
grid_search_svr.fit(X_train_loc, y_train_loc)

# Mostrar los mejores parámetros encontrados
print("Mejores hiperparámetros para SVR (goles local):")
print(grid_search_svr.best_params_)


Mejores hiperparámetros para SVR (goles local):
{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}


In [84]:
# === Entrenamiento final con los mejores parámetros ===
best_svr_model = grid_search_svr.best_estimator_
best_svr_model.fit(X_train_loc, y_train_loc)

# === Predicción sobre el conjunto de prueba ===
y_pred_loc = best_svr_model.predict(X_test_loc)


In [85]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# === Métricas de evaluación ===
rmse_loc = np.sqrt(mean_squared_error(y_test_loc, y_pred_loc))
r2_loc = r2_score(y_test_loc, y_pred_loc)

print(f"RMSE (goles local): {rmse_loc:.4f}")
print(f"R² Score (goles local): {r2_loc:.4f}")


RMSE (goles local): 1.2773
R² Score (goles local): -0.0152


In [86]:
from sklearn.inspection import permutation_importance

# Obtener los nombres de las variables predictoras
feature_names = X.columns

# Para KNN, la importancia de variables no es directa, pero podemos usar Permutation Importance

# Calculamos la importancia por permutación sobre el conjunto de test
result = permutation_importance(best_svr_model, X_test_loc, y_test_loc, n_repeats=1, random_state=42, n_jobs=-1)

# Obtenemos las 10 variables más importantes y sus importancias
importances = result.importances_mean
indices = np.argsort(importances)[::-1][:10]

print("Top 10 variables más importantes y sus pesos:")
for idx in indices:
    print(f"{feature_names[idx]}: {importances[idx]:.4f}")

# Filtramos las variables relacionadas con sustituciones
sustitucion_mask = feature_names.str.contains('cambio', case=False)
all_mask = feature_names.str.contains('e', case=False)
sustitucion_importance = abs(importances[sustitucion_mask]).sum()
all_importance = abs(importances[all_mask]).sum()
print()
print(f"\Porcentaje de importancia de variables relacionadas con sustituciones: {sustitucion_importance/all_importance:.4f}")

Top 10 variables más importantes y sus pesos:
porcentaje visitante mas 0,5 encajados en sitio: 0.0036
porcentaje visitante mas 0,5 encajados en general: 0.0035
porcentaje visitante mas 2,5 encajados en general: 0.0031
proporcion visitante corners en contra en general: 0.0028
proporcion local cambios amarillas sitio: 0.0026
proporcion visitante cambios delanteros a centrocampistas en general: 0.0025
proporcion visitante corners en contra en sitio: 0.0022
proporcion local total tiros en sitio: 0.0022
proporcion visitante goles totales en general: 0.0020
porcentaje local mas 1,5 en sitio: 0.0018

\Porcentaje de importancia de variables relacionadas con sustituciones: 0.3531


### Predicción equipo visitante

In [87]:
# === Modelo base ===
svr_model_vis = SVR()

# === Grid de hiperparámetros ===
param_grid_svr_vis = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# === Búsqueda de hiperparámetros con validación cruzada ===
grid_search_svr_vis = GridSearchCV(
    estimator=svr_model_vis,
    param_grid=param_grid_svr_vis,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

# Entrenar sobre los datos de goles visitante
grid_search_svr_vis.fit(X_train_vis, y_train_vis)

# Mostrar mejores parámetros
print("Mejores hiperparámetros para SVR (goles visitante):")
print(grid_search_svr_vis.best_params_)


Mejores hiperparámetros para SVR (goles visitante):
{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}


In [88]:
# === Entrenamiento final con los mejores parámetros ===
best_svr_model_vis = grid_search_svr_vis.best_estimator_
best_svr_model_vis.fit(X_train_vis, y_train_vis)

# === Predicción sobre el conjunto de prueba ===
y_pred_vis = best_svr_model_vis.predict(X_test_vis)


In [89]:
# === Evaluación con métricas estándar ===
rmse_vis = np.sqrt(mean_squared_error(y_test_vis, y_pred_vis))
r2_vis = r2_score(y_test_vis, y_pred_vis)

print(f"RMSE (goles visitante): {rmse_vis:.4f}")
print(f"R² Score (goles visitante): {r2_vis:.4f}")

RMSE (goles visitante): 1.0640
R² Score (goles visitante): 0.0054


In [90]:
from sklearn.inspection import permutation_importance

# Obtener los nombres de las variables predictoras
feature_names = X.columns

# Para KNN, la importancia de variables no es directa, pero podemos usar Permutation Importance

# Calculamos la importancia por permutación sobre el conjunto de test
result = permutation_importance(best_svr_model_vis, X_test_vis, y_test_vis, n_repeats=1, random_state=42, n_jobs=-1)

# Obtenemos las 10 variables más importantes y sus importancias
importances = result.importances_mean
indices = np.argsort(importances)[::-1][:10]

print("Top 10 variables más importantes y sus pesos:")
for idx in indices:
    print(f"{feature_names[idx]}: {importances[idx]:.4f}")

# Filtramos las variables relacionadas con sustituciones
sustitucion_mask = feature_names.str.contains('cambio', case=False)
all_mask = feature_names.str.contains('e', case=False)
sustitucion_importance = abs(importances[sustitucion_mask]).sum()
all_importance = abs(importances[all_mask]).sum()
print()

print(f"\Porcentaje de importancia de variables relacionadas con sustituciones: {sustitucion_importance/all_importance:.4f}")

Top 10 variables más importantes y sus pesos:
proporcion visitante corners a favor en general: 0.0034
proporcion visitante cambios amarillas en general: 0.0024
porcentaje visitante mas 0,5 marcados en general: 0.0021
proporcion visitante total tiros en general: 0.0021
proporcion visitante goles marcados en general: 0.0019
proporcion visitante posesion en general: 0.0019
proporcion visitante goles totales en general: 0.0019
porcentaje visitante perdidos en general: 0.0018
proporcion visitante posesion en sitio: 0.0017
proporcion visitante puntos en general: 0.0016

\Porcentaje de importancia de variables relacionadas con sustituciones: 0.3010
